# Regularization

Let's improve our understanding of what impacted **Titanic** passengers' chance of survival
- We will use logistic classifiers which are easy to interpret
- Remember we already did it with statsmodels in lecture "Decision Science - Logistic Regression"
- We were using `p-values` & statistical assumptions to detect which features were irrelevant / don't generalize
- This time, we will use `regularization` to detect relevant/irrelevant features based on under/overfitting criteria
- **Our goal is to compare `L1` and `L2` penalties**

## 1. We load and preprocess the data for you

In [88]:
import pandas as pd
import numpy as np

In [89]:
data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/ML_titanic_dataset_encoded.csv")

# the dataset is already one-hot-encoded
data.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [90]:
# We build X and y

y = data["survived"]
X = data.drop(columns="survived")
X.head()

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [91]:
# We MinMaxScale our features for you
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X.shape

(714, 12)

In [92]:
X_scaled

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,1.0,0.271174,0.2,0.000000,0.014151,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0.472229,0.2,0.000000,0.139136,1.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.321438,0.0,0.000000,0.015469,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.434531,0.2,0.000000,0.103644,1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.434531,0.0,0.000000,0.015713,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
709,1.0,0.484795,0.0,0.833333,0.056848,1.0,0.0,1.0,0.0,0.0,1.0,0.0
710,0.5,0.334004,0.0,0.000000,0.025374,0.0,0.0,0.0,0.0,0.0,0.0,1.0
711,0.0,0.233476,0.0,0.000000,0.058556,1.0,1.0,0.0,0.0,0.0,0.0,1.0
712,0.0,0.321438,0.0,0.000000,0.058556,0.0,1.0,0.0,0.0,1.0,0.0,0.0


## 1.  Logistic Regression without regularization

❓ Rank the features by decreasing order of importance after training a simple **non-regularized** Logistic Regression (i.e. look at the coefficients after fitting)
- Careful, `LogisticRegression` is penalized by default
- Increase `max_iter` to a larger number until model converges

In [104]:
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate 

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3)

model = LogisticRegression(max_iter=10000, penalty='none').fit(X_train, y_train)
df = pd.DataFrame(np.array((model.coef_)).T, index= X.columns, columns=['Coef'])
df.sort_values(by='Coef', ascending=False)

,Coef
sex_female,3.159689
pclass,2.638055
class_First,2.488172
who_child,0.994796
parch,0.141249
fare,-0.291889
age,-2.068427
class_Third,-2.465484
sibsp,-3.070278
embark_town_Cherbourg,-11.538550


❓How do you interpret, in plain English, the value for the coefficient `sex_female`?

<details>
    <summary>Answer</summary>

> "All other things being equal (such as age, ticket class etc...),
being a women increases your log-odds of survival by 2.67 (your coef value)"
    
> "Controling for all other explaining factors available in this dataset,
being a women increases your odds of survival by exp(2.67) = 14"

</details>


❓ What is the feature that most impacts the chances of survival according to your model?  
Fill the `top_1_feature` list below with the name of this feature

In [105]:
top_1_feature = ["embark_town_Queenstown"]

In [106]:
from nbresult import ChallengeResult
result = ChallengeResult('unregularized', top_1_feature = top_1_feature)
result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/bitazaratustra/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/bitazaratustra/code/bitazaratustra/data-challenges/05-ML/05-Model-Tuning/02-Regularization
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_unregularized.py::TestUnregularized::test_top_1 PASSED        [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/unregularized.pickle

git commit -m 'Completed unregularized step'

git push origin master


## 2.  Logistic Regression with an L2 penalty

Let's use a **Logistic model** whose log-loss has been penalized with a **L2** term to figure out the **most important features** without overfitting.  
This is the "classification" equivalent to the "Ridge" regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its features by importance (look at the coefficients)
- By "strongly regularized" we mean "more than Sklearn's default regularization factor". 
- Sklearn's default values are very useful orders of magnitudes to keep in mind for "scaled features"

In [107]:
# YOUR CODE HERE
from sklearn.linear_model import Ridge, Lasso, LogisticRegression

model2 = LogisticRegression(penalty='l2').fit(X_train, y_train)
df = pd.DataFrame((model2.coef_).T, index= X_scaled.columns)
df.sort_values(by=0,ascending=False)

,0
sex_female,2.844272
who_child,0.802068
class_First,0.584435
embark_town_Cherbourg,0.419037
parch,0.014293
fare,-0.032569
embark_town_Southampton,-0.197618
embark_town_Queenstown,-0.323264
pclass,-0.659802
class_Third,-0.735601


❓ What are the top 2 features driving chances of survival according to your model?  
Fill the `top_2_features` list below with the name of these features

In [20]:
top_2_features = ["sex_feamle", "sibsp"]

#### 🧪 Test your code below

In [21]:
from nbresult import ChallengeResult
result = ChallengeResult('ridge', top_2 = top_2_features)
result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/bitazaratustra/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/bitazaratustra/code/bitazaratustra/data-challenges/05-ML/05-Model-Tuning/02-Regularization
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_ridge.py::TestRidge::test_top2 FAILED                         [100%]

=================================== FAILURES ===================================
_____________________________ TestRidge.test_top2 ______________________________

self = <tests.test_ridge.TestRidge testMethod=test_top2>

    def test_top2(self):
        res_set = set(self.result.top_2)
        truth_set = {'sex_female', 'pclass', 'who_child', 'class_Third'}
>       self.assertTrue(len(res_set & truth_set) == 2, "Have you regularized enough? Notice that C = 1/alpha")
E       AssertionError: False is not true : Have you

## 2. Logistic Regression with an L1 penalty

This time, we'll use a logistic model whose log-loss has been penalized with a **L1** term to **filter-out the less important features**.  
This is the "classification" equivalent to the **Lasso** regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its features by importance

In [ ]:
# YOUR CODE HERE

❓ What are the features that have absolutely no impact on chances of survival, according to your L1 model?  
Fill the `zero_impact_features` list below with the name of these features; you may have to add elements to the list.

- Do you notice how some of them were "highly important" according to the non-regularized model? 
- From now on, we will always regularize our linear models!

In [ ]:
zero_impact_features = ["", "", "", ""]

#### 🧪 Test your code below

In [ ]:
from nbresult import ChallengeResult
result = ChallengeResult('lasso', zero_impact_features = zero_impact_features)
result.write()
print(result.check())

**🏁 Congratulation! Don't forget to commit and push your notebook**